In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
import json



In [ ]:
api_key = ''

In [128]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [ ]:
calibration = pd.read_csv('results/datasets/rq4/medline_1000_calibration_set.csv')

In [130]:
response = client.responses.create(
    model="gpt-4.1-mini-2025-04-14",
    input="Write a one-sentence bedtime story about a unicorn."
)


In [131]:
calibration.sort_values(by='comet_score', inplace=True)

In [132]:
bottom_250 = calibration.head(n=250)
bottom_500 =calibration.head(n=500)
top_250 = calibration.tail(n=250)
top_500 = calibration.tail(n=500)

In [133]:
top_3 = calibration.head(3)
top_3.reset_index(inplace=True)

In [134]:
top_3 = top_3[['en', 'ru']]
for i in range(len(top_3)):
    en_list =[]
    ru_list = []
    eng = top_3.iloc[i]['en']
    rus = top_3.iloc[i]['ru']
    prompt = (
        "Here is an example of a snippet of biomedical text in English and its "
        f"translation into Russian:\n\nEN: {eng}\nRU: {rus}\n\n"
        "Based on this example, be creative and generate an entirely new pair of snippets."
        "Return only valid JSON with no markdown  in the form "
        '{"en": "<novel-english-snippet>", "ru": "<russian-translation-of-novel-snippet>"}.'
    )
    response = client.responses.create(
    model="gpt-4.1-mini-2025-04-14",
    input=prompt)   

    pair = json.loads(response.output_text)
    en_list.append(pair['en'])
    ru_list.append(pair['ru'])

In [135]:
def generate_synthetic_pairs(input_pairs, synth_per_pair = 2):
    input_pairs = input_pairs[['en', 'ru']]
    en_list =[]
    ru_list = []
    failed_list = []
    for i in range(len(input_pairs)):
        for j in range(synth_per_pair):
            eng = input_pairs.iloc[i]['en']
            rus = input_pairs.iloc[i]['ru']
            prompt = (
                "Here is an example of a snippet of biomedical text in English and its "
                f"translation into Russian:\n\nEN: {eng}\nRU: {rus}\n\n"
                "Based on this example, be creative and generate an entirely new pair of snippets."
                "Return only valid JSON with no markdown in the form "
                '{"en": "<novel-english-snippet>", "ru": "<russian-translation-of-novel-snippet>"}.'
            )
            response = client.responses.create(
                model="gpt-4.1-mini-2025-04-14",
                input=prompt,
                    
            )   
            try:
                pair = json.loads(response.output_text)
                en_list.append(pair['en'])
                ru_list.append(pair['ru'])    
            except:
                failed_list.append(response)

    df = pd.DataFrame({
    'synth_en': en_list,
    'synth_ru': ru_list
    })
    return df, failed_list



In [146]:
synthetic_bottom_250, _ = generate_synthetic_pairs(bottom_250, synth_per_pair=4)

In [147]:
synthetic_bottom_250.to_csv('../results/datasets/medline_calibration_bottom_250_4x.csv')